In [ ]:
import pandas as pd
from datetime import datetime
import openpyxl
import shutil
import os
import re

In [ ]:
# Substitua 'caminho_para_o_arquivo.xlsx' pelo caminho do seu arquivo Excel
SS_EED = pd.read_excel('SS-EMS-EED-CG.xlsx')

c:\Users\EN405081\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
def map_departcode(departcode):
    if departcode == "EMS-EED-CG":
        return "DCMD"
    elif departcode == "EMS-AT-CE":
        return "DEMT"
    elif departcode == "EMS-OP-CG":
        return "DEOP"
    elif departcode == "EMS-EST-TEL":
        return "TELECON"
    elif departcode == "EMS-PROT":
        return "PROTEÇÃO"
    elif departcode == "EMS-PN2025":
        return "OBRA P/2025"
    elif departcode == "EMS-ENG":
        return "SPMA"
    elif departcode == "EMS-PRÉ-OP":
        return "PRÉ-OPERAÇÃO"
    elif departcode == "EGS-PROJ01":
        return "ENGESELT-LEVANTAMENTO"
    elif departcode == "EMS-EST-CG":
        return "ASPO"
    elif departcode == "EMS-TEL-EST":
        return "TELECON"  # Substitua "NOVO_VALOR" pelo valor desejado
    else:
        return ""


# Aplicar a função ao DataFrame
SS_EED['DEPARTAMENTO'] = SS_EED['DEPARTCODE'].apply(map_departcode)

In [ ]:
# Verificar as colunas do DataFrame
print(SS_EED.columns)

Index(['Código da Solicitação', 'Ano', 'Descrição', 'DataOcorrência',
       'Abrangencia', 'Instalação', 'Trecho', 'Localização', 'Complemento',
       'Sigla Orgao', 'SolicNome', 'DescPrioridade', 'SS-Estados',
       'Data/Hora Limite', 'Ativos', 'Qtd_Clientes', 'Espécies', 'SS-Tipos',
       'Término', 'OSNumero', 'OS-Estados', 'PlanDataInicio', 'PlanDataFinal',
       'DescOrigem', 'Condição Operativa COI', 'Descrição do Defeito', 'Lida',
       'Processo', 'DEPARTCODE', 'UF', 'Bairro', 'Logradouro', 'OS SIATT',
       'SS PRINCIPAL', 'SOMA', 'Coordenada Longitude(X)',
       'Coordenada Latitude(Y)', 'Equipamento Operando?', 'REGIONAL',
       'CÓDIGO REGIONAL', 'ESTRUTURA', 'EST PRIMÁRIA', 'EST SECUNDÁRIA',
       'NOTA', 'TIPO NOTA', 'ORDEM', 'LOCALIDADE', 'CODIGO CLIENTE',
       'CARACTERÍSTICA REGIONAL', 'Equipe de atuação', 'DEPARTAMENTO'],
      dtype='object')


In [ ]:
# Função para mapear as espécies
def map_especies(especie):
    if especie == "BANCO CAPACITOR DE DISTRIBUIÇÃO GIS":
        return "BD"
    elif especie == "LOCAL INSTAL RELIGADOR DE LINHA GIS":
        return "RD"
    elif especie == "REGULADOR DE TENSÃO GIS":
        return "RT"
    elif especie == "REGULADOR DE TENSÃO DE DISTRIBUIÇÃO GIS":
        return "RT"
    elif especie == "RELIGADOR DE REDE GIS":
        return "RD"
    else:
        return ""

# Aplicar a função ao DataFrame
SS_EED['TIPO EED'] = SS_EED['Espécies'].apply(map_especies)


In [ ]:
# Criar a coluna 'Número Operativo'
SS_EED['Número Operativo'] = SS_EED.apply(lambda row: f"{row['TIPO EED']}({row['Localização']})", axis=1)


In [ ]:
print(SS_EED[['Número Operativo']].head())

   Número Operativo
0          (108227)
1  (453RL006812819)
2         (65TF-02)
3             (CMS)
4             (nan)


In [ ]:
pim = pd.read_csv('PIM_COMPLETO_EO_TUDO.txt', delimiter=';')

# Verifique se o DataFrame foi carregado corretamente
print(pim.head())


  REGIÃO         SUPERVISÃO     LOCALIDADE COD_CIRCUITO CODIGO_INSTALACAO  \
0    SUL            NAVIRAI     MUNDO NOVO        MNO02             MNO02   
1    SUL  DOURADOS INTERIOR  RIO BRILHANTE        RBR53             RBR53   
2    SUL  DOURADOS INTERIOR  RIO BRILHANTE        RBR53         CF(16981)   
3    SUL  DOURADOS INTERIOR  RIO BRILHANTE        RBR53          CF(7183)   
4    SUL  DOURADOS INTERIOR  RIO BRILHANTE        RBR53          CF(7184)   

  DISPOSITIVO_PAI PROTEÇÃO TIPO NOME DO RELIGADOR LZ_TIPO  ...         SOMA  \
0           MNO02    MNO02   SE               NaN       U  ...     3.4 ALTO   
1           RBR53    RBR53   SE               NaN       U  ...  4.4 CRÍTICO   
2        CF(7183)    RBR53  NaN               NaN       U  ...  4.4 CRÍTICO   
3           RBR53    RBR53  NaN               NaN       U  ...  4.4 CRÍTICO   
4       CF(16981)    RBR53  NaN               NaN       R  ...  4.4 CRÍTICO   

  TOPOLOGIA      COD_SGM         COD_SIT   LATITUDE  LONGITUDE

C:\Users\EN405081\AppData\Local\Temp\ipykernel_7504\2257936149.py:1: DtypeWarning: Columns (7,10,13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  pim = pd.read_csv('PIM_COMPLETO_EO_TUDO.txt', delimiter=';')


In [ ]:
print(pim[['SUPERVISÃO','REGIÃO']].drop_duplicates())

                 SUPERVISÃO  REGIÃO
0                   NAVIRAI     SUL
1         DOURADOS INTERIOR     SUL
10                  CORUMBA   OESTE
19                 DOURADOS     SUL
24                   JARDIM   OESTE
27           NOVA ANDRADINA     SUL
51    CAMPO GRANDE INTERIOR  CENTRO
63               COSTA RICA   NORTE
68               PONTA PORA     SUL
96               AQUIDAUANA   OESTE
117                   COXIM   NORTE
199               PARANAIBA   NORTE
274            CAMPO GRANDE  CENTRO
998   CAMPO GRANDE INTERIOR   NORTE
1708                  _ERRO   _ERRO


In [ ]:
# Alterar a SUPERVISÃO para 'CAMPO GRANDE' onde SUPERVISÃO é 'CAMPO GRANDE INTERIOR' e REGIÃO é 'CENTRO'
pim.loc[(pim['SUPERVISÃO'] == 'NAVIRAI') & (pim['REGIÃO'] == 'SUL'), 'REGIÃO'] = 'LESTE'
pim.loc[(pim['SUPERVISÃO'] == 'NOVA ANDRADINA') & (pim['REGIÃO'] == 'SUL'), 'REGIÃO'] = 'LESTE'
pim.loc[(pim['SUPERVISÃO'] == 'NAVIRAI') & (pim['COD_CIRCUITO'] == 'NAV05'), 'REGIÃO'] = 'LESTE'
pim.loc[(pim['SUPERVISÃO'] == 'NAVIRAI') & (pim['COD_CIRCUITO'] == 'ELD52'), 'REGIÃO'] = 'LESTE'
pim.loc[(pim['SUPERVISÃO'] == 'NAVIRAI') & (pim['COD_CIRCUITO'] == 'ITQ01'), 'REGIÃO'] = 'LESTE'
pim.loc[(pim['SUPERVISÃO'] == 'NAVIRAI') & (pim['COD_CIRCUITO'] == 'MNO01'), 'REGIÃO'] = 'LESTE'


In [ ]:
# Alterar a SUPERVISÃO para 'CAMPO GRANDE' onde SUPERVISÃO é 'CAMPO GRANDE INTERIOR' e REGIÃO é 'CENTRO'
pim.loc[(pim['SUPERVISÃO'] == 'CAMPO GRANDE INTERIOR') & (pim['REGIÃO'] == 'CENTRO'), 'SUPERVISÃO'] = 'CAMPO GRANDE'



# Verificar as alterações
print(pim[['SUPERVISÃO', 'REGIÃO']].drop_duplicates())

# Salvar o arquivo atualizado, se necessário
pim.to_csv('PIM_COMPLETO_EO_TUDO_atualizado.txt', sep=';', index=False)



                 SUPERVISÃO  REGIÃO
0                   NAVIRAI   LESTE
1         DOURADOS INTERIOR     SUL
10                  CORUMBA   OESTE
19                 DOURADOS     SUL
24                   JARDIM   OESTE
27           NOVA ANDRADINA   LESTE
51             CAMPO GRANDE  CENTRO
63               COSTA RICA   NORTE
68               PONTA PORA     SUL
96               AQUIDAUANA   OESTE
117                   COXIM   NORTE
199               PARANAIBA   NORTE
998   CAMPO GRANDE INTERIOR   NORTE
1708                  _ERRO   _ERRO


In [ ]:
# Verificar as colunas do DataFrame
display(pim.columns)


Index(['REGIÃO', 'SUPERVISÃO', 'LOCALIDADE', 'COD_CIRCUITO',
       'CODIGO_INSTALACAO', 'DISPOSITIVO_PAI', 'PROTEÇÃO', 'TIPO',
       'NOME DO RELIGADOR', 'LZ_TIPO', 'CONJUNTO_ANEEL', 'NOME_CONJUNTO',
       'EQUIPAMENTO', 'ESTADO_NORMAL', 'FASES', 'P_PODA', 'CLIENTES', 'TRECHO',
       'CID x CID', 'CIDADE_ATENDIDA', 'KM_REDE', 'ESTRUTURAS', 'SOMA',
       'TOPOLOGIA', 'COD_SGM', 'COD_SIT', 'LATITUDE', 'LONGITUDE',
       'TRECHO_PROT', 'CID_PROT', 'PHASE', 'SPEC_NAME'],
      dtype='object')

In [ ]:
# Converter a coluna 'Número Operativo' para string
SS_EED['Número Operativo'] = SS_EED['Número Operativo'].astype(str)




In [ ]:
if 'CODIGO_INSTALACAO' in pim.columns:
    pim['CODIGO_INSTALACAO'] = pim['CODIGO_INSTALACAO'].astype(str)
else:
    print("Coluna 'CODIGO_INSTALACAO' não encontrada.")


In [ ]:
SS_EED = SS_EED.merge(
    pim[['CODIGO_INSTALACAO', 'FASES', 'TRECHO','SUPERVISÃO','NOME DO RELIGADOR','LZ_TIPO','ESTADO_NORMAL','CLIENTES', 'CID x CID','EQUIPAMENTO','REGIÃO']],
    left_on='Número Operativo',
    right_on='CODIGO_INSTALACAO',
    how='left',
    suffixes=('', '_pim')  # Adiciona sufixos para evitar duplicatas
)


In [ ]:
# Substitua 'caminho_para_o_arquivo.xlsx' pelo caminho do seu arquivo Excel
OS_EED = pd.read_excel('OS-EMS-EED-CG.xlsx')


c:\Users\EN405081\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# Verificar as colunas do DataFrame
print(OS_EED.columns)

Index(['Delta IPDT', 'OS(int)', 'OSNumero', 'Trecho', 'CodSetor',
       'Abrangência', 'Montante Abrangência', 'CodEstado', 'CodEsquema',
       'DescEsquema', 'CodPrioridade', 'PlanDataInicio', 'PlanDataFinal',
       'Equipe programada', ' TME ', 'Data Redirecionamento', ' TMD ',
       'Total (TMD+TME)', 'Observação', 'DataCriação', 'DataDeadline',
       'DataPrevista', 'Titulo Lote', 'Código do Lote', 'Quantidade clientes',
       'Código descrição contrato', 'Cod. Contrato Ext.',
       'Sigla Atividade Contrato', 'Descrição atividade contrato',
       'CodFamília', 'Familias', 'CodAtivo', 'Descrição Ativo', 'SS Associada',
       'Processo', 'Impedimento', 'UF', 'Localidade', 'Logradouro', 'Bairro',
       'Complemento de Localização', 'NumSérie', 'EmpresaProprietária',
       'PlanMatrícula', 'CodLocalização', 'Total Material', 'Total Recurso',
       'Valor Total', 'Código Externo', 'Origem da criação', 'Plano Trabalho',
       'OS Original', 'Num. Obra', 'Num. Obra Ativo', '

In [ ]:

print(SS_EED[['OSNumero']])

                      OSNumero
0                          NaN
1                          NaN
2       EMS-SE-CE  000473/2019
3                          NaN
4                          NaN
...                        ...
27500                      NaN
27501   EMS-AT-CE  003500/2024
27502  EMS-EST-CG  000037/2024
27503   EMS-AT-CE  003419/2024
27504                      NaN

[27505 rows x 1 columns]


In [ ]:

print(OS_EED[['OSNumero']])

                    OSNumero
0    EMS-EED-CG  000412/2023
1    EMS-EED-CG  000140/2024
2    EMS-EED-CG  000184/2024
3    EMS-EED-CG  000362/2023
4    EMS-EED-CG  000287/2023
..                       ...
388  EMS-EED-CG  000039/2020
389  EMS-EED-CG  000032/2020
390  EMS-EED-CG  000020/2020
391  EMS-EED-CG  000010/2020
392  EMS-EED-CG  000002/2020

[393 rows x 1 columns]


In [ ]:
# Print the column names to verify
print(SS_EED.columns)
print(OS_EED.columns)



Index(['Código da Solicitação', 'Ano', 'Descrição', 'DataOcorrência',
       'Abrangencia', 'Instalação', 'Trecho', 'Localização', 'Complemento',
       'Sigla Orgao', 'SolicNome', 'DescPrioridade', 'SS-Estados',
       'Data/Hora Limite', 'Ativos', 'Qtd_Clientes', 'Espécies', 'SS-Tipos',
       'Término', 'OSNumero', 'OS-Estados', 'PlanDataInicio', 'PlanDataFinal',
       'DescOrigem', 'Condição Operativa COI', 'Descrição do Defeito', 'Lida',
       'Processo', 'DEPARTCODE', 'UF', 'Bairro', 'Logradouro', 'OS SIATT',
       'SS PRINCIPAL', 'SOMA', 'Coordenada Longitude(X)',
       'Coordenada Latitude(Y)', 'Equipamento Operando?', 'REGIONAL',
       'CÓDIGO REGIONAL', 'ESTRUTURA', 'EST PRIMÁRIA', 'EST SECUNDÁRIA',
       'NOTA', 'TIPO NOTA', 'ORDEM', 'LOCALIDADE', 'CODIGO CLIENTE',
       'CARACTERÍSTICA REGIONAL', 'Equipe de atuação', 'DEPARTAMENTO',
       'TIPO EED', 'Número Operativo', 'CODIGO_INSTALACAO', 'FASES', 'TRECHO',
       'SUPERVISÃO', 'NOME DO RELIGADOR', 'LZ_TIPO', 'EST

In [ ]:
# Check for missing values in OSNumero columns
print(SS_EED['OSNumero'].isnull().sum())  # Count missing values in SS_EED
print(OS_EED['OSNumero'].isnull().sum())  # Count missing values in OS_EED



8598
0


In [ ]:
# Strip leading/trailing whitespace in OSNumero columns
SS_EED['OSNumero'] = SS_EED['OSNumero'].str.strip()
OS_EED['OSNumero'] = OS_EED['OSNumero'].str.strip()


In [ ]:
SS_EED = SS_EED.merge(
    OS_EED[['OSNumero', 'Num. Obra', 'PlanDataInicio', 'DataCriação']],
    on='OSNumero',  # Common column for merging
    how='left',
    suffixes=('', 'OS')
)


In [ ]:

print(SS_EED[['PlanDataInicio']])

       PlanDataInicio
0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
...               ...
27500             NaN
27501             NaN
27502             NaN
27503             NaN
27504             NaN

[27505 rows x 1 columns]


In [ ]:

# Ler o arquivo Excel, pulando as três primeiras linhas e definindo o cabeçalho na quarta linha
GS = pd.read_excel(
    'GS.xlsx',
    skiprows=4,  # Ignorar as três primeiras linhas
    header=0     # Definir a quarta linha como o cabeçalho
)

# Verificar os primeiros registros para garantir que os dados estão corretos
print(GS.head())
print(GS.columns)

        Obra  Dep. Obra      Num. OS  Região    Supervisão  \
0  102305536        NaN  128780737.0  CENTRO  CAMPO GRANDE   
1  102306866        NaN   87086468.0  CENTRO  CAMPO GRANDE   
2  102403272        NaN  268379224.0  CENTRO  CAMPO GRANDE   
3  502400565        NaN  217638469.0   LESTE       NAVIRAI   
4  152100245        NaN   45989255.0   OESTE        JARDIM   

           Localidade                      Descrição   Empreiteira Status_GS  \
0  RIBAS DO RIO PARDO     CIA x TERCEIROS VEGETATIVO    COMPEL-DOU  EXECUTAR   
1        CAMPO GRANDE  CIA x 3º - Carga > 50kW - ERD  ENGELMIG-CGR  EXECUTAR   
2        CAMPO GRANDE     CIA x TERCEIROS VEGETATIVO  ENGELMIG-CGR  EXECUTAR   
3              TACURU  CIA x 3º - Carga > 50kW - ERD       JVP-NAV  EXECUTAR   
4          BELA VISTA  SEGURANÇA: MAP. RISCO EMPENHO       JVP-JAR  EXECUTAR   

   Ranking  ...    Regulado Inicio Físico Data Status  Dias suspensão  \
0        0  ...    Anexo IV    2023-11-13  2023-11-13               0   


In [ ]:
GS.columns = GS.columns.str.strip()

In [ ]:
SS_EED.columns = SS_EED.columns.str.strip()

In [ ]:
# Realizar o merge com os nomes de coluna corretos
SS_EED = SS_EED.merge(
    GS[['Obra', 'Empreiteira', 'Status_GS', 'Inicio Físico']],
    left_on='Num. Obra',         # Nome da coluna no SS_EED
    right_on='Obra',   # Nome da coluna no GS
    how='left',
    suffixes=('', '_Obra')  # Adiciona sufixos para evitar duplicatas
)

In [ ]:
# Substitua 'Nome_da_Tabela' pelo nome da aba que você deseja importar
BASE_ADMs = pd.read_excel('BASE_ADMS.xlsx', 
                          sheet_name='ADMS')



In [ ]:
# Criar a coluna 'Número Operativo'
BASE_ADMs['Número Operativo'] = BASE_ADMs.apply(lambda row: f"{row['TIPO']}({row['Equipamento']})", axis=1)

In [ ]:
# Checking columns in SS_EED and BASE_ADMs
print(SS_EED.columns)
print(BASE_ADMs.columns)

# Strip leading/trailing spaces if necessary
SS_EED.columns = SS_EED.columns.str.strip()
BASE_ADMs.columns = BASE_ADMs.columns.str.strip()

# Try the merge again
SS_EED = SS_EED.merge(
    BASE_ADMs[['MODELO', 'Nome', 'Master', 'Número Operativo']],
    on='Número Operativo',
    how='left'
)


Index(['Código da Solicitação', 'Ano', 'Descrição', 'DataOcorrência',
       'Abrangencia', 'Instalação', 'Trecho', 'Localização', 'Complemento',
       'Sigla Orgao', 'SolicNome', 'DescPrioridade', 'SS-Estados',
       'Data/Hora Limite', 'Ativos', 'Qtd_Clientes', 'Espécies', 'SS-Tipos',
       'Término', 'OSNumero', 'OS-Estados', 'PlanDataInicio', 'PlanDataFinal',
       'DescOrigem', 'Condição Operativa COI', 'Descrição do Defeito', 'Lida',
       'Processo', 'DEPARTCODE', 'UF', 'Bairro', 'Logradouro', 'OS SIATT',
       'SS PRINCIPAL', 'SOMA', 'Coordenada Longitude(X)',
       'Coordenada Latitude(Y)', 'Equipamento Operando?', 'REGIONAL',
       'CÓDIGO REGIONAL', 'ESTRUTURA', 'EST PRIMÁRIA', 'EST SECUNDÁRIA',
       'NOTA', 'TIPO NOTA', 'ORDEM', 'LOCALIDADE', 'CODIGO CLIENTE',
       'CARACTERÍSTICA REGIONAL', 'Equipe de atuação', 'DEPARTAMENTO',
       'TIPO EED', 'Número Operativo', 'CODIGO_INSTALACAO', 'FASES', 'TRECHO',
       'SUPERVISÃO', 'NOME DO RELIGADOR', 'LZ_TIPO', 'EST

In [ ]:


# Caminho da pasta de origem
# origem = r'\\fs-bi\EMS\DTEC\DCMD\DW_DCMD\SICEC'

# Caminho da pasta de destino
# destino = r'C:\Users\EN405081\Projetos Python\BI_EQUIPAMENTOS'

# Nomes dos arquivos que você quer copiar
# arquivos_para_transferir = [
   #  'SICEC_C111_Estoque_Materiais.txt', 
   #  'SICEC_C211_Movimentacao_Materiais_Definitivo_2024.txt',
   #  'SICEC_C212_Movimentacao_Materiais_Temporario_2024.txt'
# ]

# Copia cada arquivo para a pasta de destino, substituindo o existente
# for arquivo in arquivos_para_transferir:
  #   caminho_completo_origem = os.path.join(origem, arquivo)
    # caminho_completo_destino = os.path.join(destino, arquivo)
    
    # Verifica se o arquivo de origem existe antes de copiar
    # if os.path.exists(caminho_completo_origem):
      #   shutil.copy2(caminho_completo_origem, caminho_completo_destino)
       #  print(f"{arquivo} copiado com sucesso!")
    # else:
      #   print(f"Arquivo não encontrado: {arquivo}")

# print("Processo de cópia concluído!")






In [ ]:
SIAGO_C611_Status_Conc_Fisica = pd.read_csv('SIAGO_C611_Status_Conc_Fisica.txt', delimiter='\t', encoding='latin1')
SIAGO_C611_Status_Fisc_Aprovada = pd.read_csv('SIAGO_C611_Status_Fisc_Aprovada.txt', delimiter='\t', encoding='latin1')
SIAGO_C611_Status_Fisc_Reprovada = pd.read_csv('SIAGO_C611_Status_Fisc_Reprovada.txt', delimiter='\t', encoding='latin1')

# Unir as bases de dados verticalmente
SIAGO_C611 = pd.concat([SIAGO_C611_Status_Conc_Fisica, SIAGO_C611_Status_Fisc_Aprovada, SIAGO_C611_Status_Fisc_Reprovada], axis=0, ignore_index=True)

In [ ]:
display(SIAGO_C611)

,num_obra,cod_regional,tipo_servico,localidade,status_atual,data_hora,sgl_executor,dscgrp_obra,num_projeto_logico,mt_especifico,descricao,num_obra_dependencia,data_ultima_suspensao,motivo_suspensão_projeto,cod_suspensão_projeto,ind_origem_sol,ind_geracao_distribuida,num_projeto_sigco,dsc_prazo_conclusao,ind_nivel_corrosao
0,101800001,10,INSTALAÇÃO DE BANCO DE CAPACITOR,CAMPO GRANDE,ENCERRAMENTO:ENCERRAMENTO TÉCNICO,17/12/2018 15:10:29,EN405080,Própria,NaN,SIM,ORDEM DE IMOBILIZAÇÃO,NaN,NaN,NaN,NaN,NaN,N,22777.0,Outros,NaN
1,101800007,10,OUTROS,CAMPO GRANDE,ENCERRAMENTO:ENCERRAMENTO TÉCNICO,27/06/2018 00:00:00,EN404406,Própria,NaN,NÃO,ORDEM DE IMOBILIZAÇÃO,NaN,NaN,NaN,NaN,NaN,N,11826.0,60 dias,NaN
2,101800009,10,CONEXÃO REDE com PE - GRUPO B,AGUA CLARA,ENCERRAMENTO:ENCERRAMENTO TÉCNICO,10/09/2019 14:09:59,EN405262,Cia X 3º,NaN,NÃO,ORDEM DE SERVIÇO,NaN,NaN,NaN,NaN,NaN,N,12181.0,120 dias,NaN
3,101800010,10,CONEXÃO REDE com PE - GRUPO B,AGUA CLARA,INDEFERIDA,21/07/2021 14:14:38,EN405262,Cia X 3º,NaN,NÃO,ORDEM DE DESATIVAÇÃO,NaN,NaN,NaN,NaN,NaN,N,12181.0,120 dias,NaN
4,101800011,10,MELHORIA DE REDE AUMENTO DE CARGA,CAMPO GRANDE,ENCERRAMENTO:ENCERRAMENTO TÉCNICO,27/06/2018 00:00:00,EN404406,Cia X 3º,NaN,SIM,ORDEM DE IMOBILIZAÇÃO,NaN,NaN,NaN,NaN,NaN,N,11827.0,120 dias,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340857,712101755,71,MANUTENÇÃO LD/RD,RIO VERDE DE MATO GROSSO,ENCERRAMENTO:ENCERRAMENTO TÉCNICO,19/05/2023 16:45:59,EN405262,Manutenção,270885.0,NÃO,ORDEM DE IMOBILIZAÇÃO,NaN,NaN,NaN,NaN,NaN,N,8945.0,60 dias,NaN
340858,712200222,71,MANUTENÇÃO LD/RD,SAO GABRIEL DO OESTE,ENCERRAMENTO:ENCERRAMENTO TÉCNICO,28/10/2022 17:35:09,SCGCC,Manutenção,303494.0,NÃO,ORDEM DE IMOBILIZAÇÃO,NaN,NaN,NaN,NaN,NaN,N,9171.0,60 dias,NaN
340859,712300077,71,MANUTENÇÃO LD/RD,RIO VERDE DE MATO GROSSO,ENCERRAMENTO:FISCALIZAÇÃO REPROVADA,23/02/2023 16:25:32,EN404316,Manutenção,NaN,NÃO,DESPESA,NaN,NaN,NaN,NaN,NaN,N,24356.0,60 dias,NaN
340860,712300078,71,MANUTENÇÃO LD/RD,COXIM,ENCERRAMENTO:FISCALIZAÇÃO APROVADA,10/04/2023 16:19:47,EN404316,Manutenção,NaN,NÃO,DESPESA,NaN,NaN,NaN,NaN,NaN,N,24356.0,60 dias,NaN


In [ ]:
# Realizar o merge com os nomes de coluna corretos
SS_EED = SS_EED.merge(
    SIAGO_C611[[ 'num_obra','status_atual']],
    left_on='Obra',         # Nome da coluna no SS_EED
    right_on='num_obra',   # Nome da coluna no GS
    how='left',
    suffixes=('', '_Obra')  # Adiciona sufixos para evitar duplicatas
)

In [ ]:

SIAGO_C37_Obras_e_OSs_Associadas = pd.read_csv('SIAGO_C37_Obras_e_OSs_Associadas.txt', delimiter='\t', encoding='latin1')


C:\Users\EN405081\AppData\Local\Temp\ipykernel_7504\2813816081.py:1: DtypeWarning: Columns (4,5,17) have mixed types. Specify dtype option on import or set low_memory=False.
  SIAGO_C37_Obras_e_OSs_Associadas = pd.read_csv('SIAGO_C37_Obras_e_OSs_Associadas.txt', delimiter='\t', encoding='latin1')


In [ ]:
SIAGO_C37_Obras_e_OSs_Associadas.head(5)

,regional,num_obra,num_projeto_logico,num_os,dsc_os,numos,cod_localidade,localidade,controle,ind_abertura_projeto,...,cod_circuito,tipo_servico,dsc_tipo_projeto,num_obra_depencencia,dsc_situacao_sistema_gis,dsc_status,dsc_situacao_sistema_sgm,idt_os_sgm,num_coordenada_x_uc,num_coordenada_y_uc
0,AQUIDAUANA - Manutenção,NaN,NaN,624283.0,EMS-RD-AQ 000005/2018,NaN,NaN,NaN,0,N,...,NaN,NaN,NaN,NaN,NaN,NaN,EXECUTADA,12.0,NaN,NaN
1,COXIM - Manutenção,NaN,NaN,624299.0,EMS-RD-CX 000002/2018,NaN,NaN,NaN,0,S,...,NaN,NaN,NaN,NaN,NaN,NaN,EXECUTADA,28.0,NaN,NaN
2,COXIM - Manutenção,NaN,NaN,624301.0,EMS-RD-CX 000004/2018,NaN,NaN,NaN,0,S,...,NaN,NaN,NaN,NaN,NaN,NaN,EXECUTADA,30.0,NaN,NaN
3,COXIM - Manutenção,NaN,NaN,624303.0,ECO-PCM-CX 000005/2018,NaN,NaN,NaN,0,S,...,NaN,NaN,NaN,NaN,NaN,NaN,CANCELADA,32.0,NaN,NaN
4,COXIM - Manutenção,NaN,NaN,624304.0,ECO-PCM-CX 000006/2018,NaN,NaN,NaN,0,S,...,NaN,NaN,NaN,NaN,NaN,NaN,CANCELADA,33.0,NaN,NaN


In [ ]:


# Supondo que você já tenha o DataFrame SS_EED carregado e manipulado

# Defina o caminho e o nome do arquivo Excel de saída
caminho_arquivo = 'SS_EED.xlsx'

# Exportar o DataFrame para um arquivo Excel
SS_EED.to_excel(caminho_arquivo, index=False, sheet_name='Dados')

print(f"Arquivo Excel salvo em: {caminho_arquivo}")


Arquivo Excel salvo em: SS_EED.xlsx


In [ ]:
# Agrupar por 'EQUIPAMENTO' e contar os valores únicos de 'CODIGO_INSTALACAO'
contagem_parque = pim.groupby('EQUIPAMENTO')['CODIGO_INSTALACAO'].nunique().reset_index()

# Renomear a coluna de contagem para algo mais descritivo
contagem_parque.rename(columns={'CODIGO_INSTALACAO': 'Quantidade_Instalacoes'}, inplace=True)

print(contagem_parque)

                             EQUIPAMENTO  Quantidade_Instalacoes
0                       AL - Alimentador                     425
1   BD - Banco Capacitor de Distribuição                     526
2        CE - Seccionalizador Eletrônico                     293
3                        CF - Chave Faca                   12697
4                      CL - Chave Lâmina                     846
5                CS - Chave Seccionadora                       6
6                    CT - Chave Tripolar                     170
7          FR - Chave Fusível Religadora                    1659
8                     FU - Chave Fusível                   35134
9                RD - Religador de Linha                    1691
10              RT - Regulador de Tensão                     285
11                      TA - Aterramento                       1
12                     TD - Distribuição                   93703


In [ ]:

# Remover espaços em branco dos nomes das colunas, se necessário
pim.columns = pim.columns.str.strip()

# Agrupar por 'EQUIPAMENTO' e contar os valores únicos de 'CODIGO_INSTALACAO'
contagem_parque = pim.groupby('EQUIPAMENTO')['CODIGO_INSTALACAO'].nunique().reset_index()

# Renomear a coluna de contagem para algo mais descritivo
contagem_parque.rename(columns={'CODIGO_INSTALACAO': 'Quantidade_Instalacoes'}, inplace=True)

# Filtrar os equipamentos específicos
equipamentos_especificos = ['BD - Banco Capacitor de Distribuição', 'RD - Religador de Linha', 'RT - Regulador de Tensão', 'TA - Aterramento']
contagem_filtrada = contagem_parque[contagem_parque['EQUIPAMENTO'].isin(equipamentos_especificos)]

print(contagem_filtrada)

                             EQUIPAMENTO  Quantidade_Instalacoes
1   BD - Banco Capacitor de Distribuição                     526
9                RD - Religador de Linha                    1691
10              RT - Regulador de Tensão                     285
11                      TA - Aterramento                       1


In [ ]:
SS_EED.columns = SS_EED.columns.str.strip()

# Lista de equipamentos específicos que queremos analisar
equipamentos_especificos = [
    'BD - Banco Capacitor de Distribuição',
    'RD - Religador de Linha',
    'RT - Regulador de Tensão',
    'TA - Aterramento'
]

# Filtrar o DataFrame com base nos critérios fornecidos e nos equipamentos específicos
equipamentos_falharam = SS_EED[(SS_EED['SS-Estados'] == 'SS PENDENTE') &
                               (SS_EED['DEPARTAMENTO'] == 'DCMD') &
                               (SS_EED['Equipamento Operando?'] == 'Não') &
                               (SS_EED['EQUIPAMENTO'].isin(equipamentos_especificos))]
equssddd = equipamentos_falharam[equipamentos_falharam['EQUIPAMENTO'].isin(equipamentos_especificos)]

# Contar o número de ocorrências de cada tipo de equipamento
contagem_equipamentos = equipamentos_falharam['EQUIPAMENTO'].value_counts()

print(contagem_equipamentos)


EQUIPAMENTO
BD - Banco Capacitor de Distribuição    44
RD - Religador de Linha                 28
RT - Regulador de Tensão                20
Name: count, dtype: int64


In [ ]:
# Converta a série de contagem de falhas em DataFrame
contagem_falhas_df = contagem_equipamentos.reset_index()
contagem_falhas_df.columns = ['EQUIPAMENTO', 'Quantidade_Falhas']

# Mesclar os DataFrames de contagem de falhas e contagem de instalações
tabela_consolidada = pd.merge(contagem_filtrada, contagem_falhas_df, on='EQUIPAMENTO', how='left')

# Calcular a taxa de falha
tabela_consolidada['Taxa_Falha'] = tabela_consolidada['Quantidade_Falhas'] / tabela_consolidada['Quantidade_Instalacoes']

# Substituir valores NaN na taxa de falha por 0 (se necessário)
tabela_consolidada['Taxa_Falha'] = tabela_consolidada['Taxa_Falha'].fillna(0)

display(tabela_consolidada)

,EQUIPAMENTO,Quantidade_Instalacoes,Quantidade_Falhas,Taxa_Falha
0,BD - Banco Capacitor de Distribuição,526,44.0,0.083650
1,RD - Religador de Linha,1691,28.0,0.016558
2,RT - Regulador de Tensão,285,20.0,0.070175
3,TA - Aterramento,1,NaN,0.000000


In [ ]:
# Salvar o DataFrame final em um arquivo Excel ou CSV
tabela_consolidada.to_excel('taxa_falha.xlsx', index=False)

In [ ]:

# Supondo que o DataFrame 'pim' já esteja carregado e as colunas estejam corretamente nomeadas
pim.columns = pim.columns.str.strip()

# Agrupar por 'SUPERVISÃO' e 'EQUIPAMENTO' e contar a quantidade de ocorrências
contagem_por_supervisao = pim.groupby(['REGIÃO','SUPERVISÃO', 'EQUIPAMENTO']).size().reset_index(name='Quantidade')

# Filtrar os equipamentos específicos se necessário
equipamentos_especificos = ['BD - Banco Capacitor de Distribuição', 'RD - Religador de Linha', 'RT - Regulador de Tensão', 'TA - Aterramento']
contagem_filtrada_supervisao = contagem_por_supervisao[contagem_por_supervisao['EQUIPAMENTO'].isin(equipamentos_especificos)]

# Exibir o DataFrame filtrado
print(contagem_filtrada_supervisao)



     REGIÃO             SUPERVISÃO                           EQUIPAMENTO  \
1    CENTRO           CAMPO GRANDE  BD - Banco Capacitor de Distribuição   
9    CENTRO           CAMPO GRANDE               RD - Religador de Linha   
10   CENTRO           CAMPO GRANDE              RT - Regulador de Tensão   
13    LESTE                NAVIRAI  BD - Banco Capacitor de Distribuição   
21    LESTE                NAVIRAI               RD - Religador de Linha   
22    LESTE                NAVIRAI              RT - Regulador de Tensão   
25    LESTE         NOVA ANDRADINA  BD - Banco Capacitor de Distribuição   
32    LESTE         NOVA ANDRADINA               RD - Religador de Linha   
33    LESTE         NOVA ANDRADINA              RT - Regulador de Tensão   
36    NORTE  CAMPO GRANDE INTERIOR  BD - Banco Capacitor de Distribuição   
43    NORTE  CAMPO GRANDE INTERIOR               RD - Religador de Linha   
44    NORTE  CAMPO GRANDE INTERIOR              RT - Regulador de Tensão   
47    NORTE 

In [ ]:

# Supondo que os DataFrames 'pim' e 'SS_EED' já estejam carregados

# Definir a lista de equipamentos específicos
equipamentos_especificos = [
    'BD - Banco Capacitor de Distribuição',
    'RD - Religador de Linha',
    'RT - Regulador de Tensão',
    'TA - Aterramento'
]


In [ ]:

# 1. Filtrar o DataFrame 'pim' para equipamentos específicos
pim.columns = pim.columns.str.strip()
pim_filtrado = pim[pim['EQUIPAMENTO'].isin(equipamentos_especificos)]



In [ ]:
# Agrupar por 'SUPERVISÃO' e 'EQUIPAMENTO' e contar a quantidade de instalações
contagem_por_supervisao_instalacoes = pim_filtrado.groupby(['REGIÃO','SUPERVISÃO', 'EQUIPAMENTO']).size().reset_index(name='Quantidade_Instalacoes')


In [ ]:

# 2. Filtrar o DataFrame 'SS_EED' para equipamentos específicos e falhas
SS_EED.columns = SS_EED.columns.str.strip()
equipamentos_falharam = SS_EED[(SS_EED['SS-Estados'] == 'SS PENDENTE') &
                               (SS_EED['DEPARTAMENTO'] == 'DCMD') &
                               (SS_EED['Equipamento Operando?'] == 'Não') &
                               (SS_EED['EQUIPAMENTO'].isin(equipamentos_especificos))]



In [ ]:
print(equipamentos_falharam['Equipamento Operando?'])

19138    Não
20843    Não
20849    Não
21174    Não
21353    Não
        ... 
27425    Não
27436    Não
27439    Não
27451    Não
27505    Não
Name: Equipamento Operando?, Length: 92, dtype: object


In [ ]:
# Contar a quantidade de falhas por SUPERVISÃO e EQUIPAMENTO
contagem_por_supervisao_falhas = equipamentos_falharam.groupby(['SUPERVISÃO', 'EQUIPAMENTO']).size().reset_index(name='Quantidade_Falhas')



In [ ]:
# 3. Mesclar os dados de falhas com os dados de instalações
tabela_consolidada = pd.merge(contagem_por_supervisao_instalacoes, contagem_por_supervisao_falhas, on=['SUPERVISÃO', 'EQUIPAMENTO'], how='left')


In [ ]:

# 4. Calcular a taxa de falha
tabela_consolidada['Quantidade_Falhas'] = tabela_consolidada['Quantidade_Falhas'].fillna(0)
tabela_consolidada['Taxa_Falha'] = tabela_consolidada['Quantidade_Falhas'] / tabela_consolidada['Quantidade_Instalacoes']



In [ ]:

# 5. Exibir o DataFrame com a taxa de falha
display(tabela_consolidada)


,REGIÃO,SUPERVISÃO,EQUIPAMENTO,Quantidade_Instalacoes,Quantidade_Falhas,Taxa_Falha
0,CENTRO,CAMPO GRANDE,BD - Banco Capacitor de Distribuição,179,13.0,0.072626
1,CENTRO,CAMPO GRANDE,RD - Religador de Linha,476,3.0,0.006303
2,CENTRO,CAMPO GRANDE,RT - Regulador de Tensão,53,2.0,0.037736
3,LESTE,NAVIRAI,BD - Banco Capacitor de Distribuição,37,2.0,0.054054
4,LESTE,NAVIRAI,RD - Religador de Linha,103,3.0,0.029126
5,LESTE,NAVIRAI,RT - Regulador de Tensão,26,1.0,0.038462
6,LESTE,NOVA ANDRADINA,BD - Banco Capacitor de Distribuição,33,0.0,0.000000
7,LESTE,NOVA ANDRADINA,RD - Religador de Linha,107,1.0,0.009346
8,LESTE,NOVA ANDRADINA,RT - Regulador de Tensão,35,1.0,0.028571
9,NORTE,CAMPO GRANDE INTERIOR,BD - Banco Capacitor de Distribuição,3,0.0,0.000000


In [ ]:

# Opcional: salvar em um arquivo Excel ou CSV
tabela_consolidada.to_excel('taxa_falha_por_supervisao.xlsx', index=False)

In [ ]:

# Caminho da pasta de origem
#origem = r'C:\Users\EN405081\Projetos Python'

# Caminho da pasta de destino
#destino = r'C:\Users\EN405081\OneDrive - Energisa\Área de Trabalho\BI_EQUIPAMENTOS'

# Nomes dos arquivos que você quer transferir
#arquivos_para_transferir = ['taxa_falha_por_supervisao.xlsx', 'taxa_falha.xlsx']

# Transfere cada arquivo para a pasta de destino
#for arquivo in arquivos_para_transferir:
    #caminho_completo_origem = os.path.join(origem, arquivo)
    #caminho_completo_destino = os.path.join(destino, arquivo)
    #shutil.move(caminho_completo_origem, caminho_completo_destino)

#print("Transferência concluída!")



In [ ]:

# Substitua 'caminho_para_o_arquivo.xlsx' pelo caminho do seu arquivo Excel
# Substitua 'Nome_da_Tabela' pelo nome da aba que você deseja importar
RESUMO_EQ = pd.read_excel('Resumo Equipamentos.xlsx', sheet_name='Equipamentos',
    skiprows=1,  # Ignorar as três primeiras linhas
    header=0     # Definir a quarta linha como o cabeçalho)
)

# Verificar os primeiros registros para garantir que os dados estão corretos



c:\Users\EN405081\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell BK1663 is marked as a date but the serial value 8219161 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
c:\Users\EN405081\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
# Substitua 'caminho_para_o_arquivo.xlsx' pelo caminho do seu arquivo Excel
# Substitua 'Nome_da_Tabela' pelo nome da aba que você deseja importar
Dicionario = pd.read_excel('Resumo Equipamentos.xlsx', sheet_name='Dicionario',
    skiprows=0,  # Ignorar as três primeiras linhas
    header=0     # Definir a quarta linha como o cabeçalho)
)

# Verificar os primeiros registros para garantir que os dados estão corretos


In [ ]:
Dicionario.head(5)

,Tipo de equipamento,TESTE,Codigo Material,descrição do material,Simplificado,Filtro BI,CLASSE,STATUS
0,VIGA PARA RT ACIMA DE 300A,NaN,691002,"VIGA METALICA PERFIL U ACO GALV 6000MM 254X69,...",VIGA - 6000MM,VIGA,REGULADOR,PADRONIZADO
1,VIGA PARA RT ACIMA DE 300A,NaN,690940,"VIGA METALICA PERFIL U ACO GALV 900MM 101,60X4...",VIGA - 900MM,VIGA,REGULADOR,PADRONIZADO
2,VIGA - 800MM,NaN,5045,VIGA COMUM RT 800MM,VIGA - 800MM,VIGA,NaN,NaN
3,VIGA - 6000MM,NaN,5046,VIGA COMUM RT 6000MM,VIGA - 6000MM,VIGA,NaN,NaN
4,TRANSFORMADOR DE PONTÊNCIAL,NaN,690032,"TRANSF POTEN SED IND EXT 13,8KV 115V/115:R3V 6...","TP - 13,8KV",TP,RELIGADOR,PADRONIZADO


In [ ]:
SIAGO_C622 = pd.read_csv('SIAGO_C622_Prev_x_Real_por_Obra.txt', delimiter='\t', encoding='latin1')


C:\Users\EN405081\AppData\Local\Temp\ipykernel_7504\1905306857.py:1: DtypeWarning: Columns (23,25,26,35) have mixed types. Specify dtype option on import or set low_memory=False.
  SIAGO_C622 = pd.read_csv('SIAGO_C622_Prev_x_Real_por_Obra.txt', delimiter='\t', encoding='latin1')


In [ ]:
display(SIAGO_C622.columns)

Index(['status_dsc_status', 'nome', 'sigla', 'val_material', 'dscsit_mat',
       'cod_empreiteira', 'rzo_social', 'num_obra', 'tip_envolvido',
       'cod_envolvido', 'nome_envolvido', 'mat_fiscal', 'tip_servico',
       'cd_tipo_servico_descricao', 'tip_obra', 'cd_tipo_obra_descricao',
       'tip_projeto', 'cd_tipo_projeto_descricao', 'cod_item_obra',
       'cd_material_descricao', 'und_medida', 'qtd_item_prev', 'qtd_rma',
       'qtd_outros', 'qtd_dma', 'qtd_justificada', 'qtdmat_apl',
       'qtdmat_apl_fsc', 'cc_tipo_movimento', 'cc_cod_movimento',
       'mat_emprestimo', 'rma_movmat', 'dma_movmat', 'qtd_nrm',
       'qtd_nf_empreiteira', 'qtd_nf_outros', 'vlr_uc_prv', 'vlr_uc_rea',
       'vlr_com_prv', 'vlr_com_rea', 'vlr_ca_prv', 'vlr_ca_rea',
       'uc_com_controlado', 'ind_nivel_corrosao'],
      dtype='object')

In [ ]:
display(SIAGO_C622['cod_item_obra'])

0         90198
1          2886
2          2955
3          3999
4          4875
          ...  
469269    42829
469270    42829
469271    42829
469272    42829
469273    42829
Name: cod_item_obra, Length: 469274, dtype: int64

In [ ]:
SIAGO_C622 = SIAGO_C622.merge(
    Dicionario[['Codigo Material', 'Simplificado']],
    left_on='cod_item_obra',
    right_on='Codigo Material',
    how='left'
)

In [ ]:
GS.head(5)

,Obra,Dep. Obra,Num. OS,Região,Supervisão,Localidade,Descrição,Empreiteira,Status_GS,Ranking,...,Regulado,Inicio Físico,Data Status,Dias suspensão,Detalhamento da Obra,Respostas,Medidor,Latitute,Longitude,Cod IBGE
0,102305536,NaN,128780737.0,CENTRO,CAMPO GRANDE,RIBAS DO RIO PARDO,CIA x TERCEIROS VEGETATIVO,COMPEL-DOU,EXECUTAR,0,...,Anexo IV,2023-11-13,2023-11-13,0,O PROJETO CONTEMPLA A EXTENSÃO DE 5.840M DE RE...,NaN,NECESSITA INSTALAÇÃO,-21.413025,-53.406458,5007109
1,102306866,NaN,87086468.0,CENTRO,CAMPO GRANDE,CAMPO GRANDE,CIA x 3º - Carga > 50kW - ERD,ENGELMIG-CGR,EXECUTAR,1,...,Reclamação,NaT,2024-11-13,254,CONEXÃO REDE p-LIG NOVA GRUPO A,NaN,NaN,-20.587169,-54.539995,5002704
2,102403272,NaN,268379224.0,CENTRO,CAMPO GRANDE,CAMPO GRANDE,CIA x TERCEIROS VEGETATIVO,ENGELMIG-CGR,EXECUTAR,2,...,Anexo IV,NaT,2024-09-20,0,O PROJETO CONTEMPLA A INSTALAÇÃO DE 1 POSTE CO...,NaN,NECESSITA INSTALAÇÃO,-20.368604,-54.675658,5002704
3,502400565,NaN,217638469.0,LESTE,NAVIRAI,TACURU,CIA x 3º - Carga > 50kW - ERD,JVP-NAV,EXECUTAR,3,...,Anexo IV,2024-07-29,2024-07-29,0,EXTENSÃO DE 130 METROS DE CONDUTOR TRIFÁSICO C...,NaN,NECESSITA INSTALAÇÃO,-23.636800,-55.022528,5007950
4,152100245,NaN,45989255.0,OESTE,JARDIM,BELA VISTA,SEGURANÇA: MAP. RISCO EMPENHO,JVP-JAR,EXECUTAR,4,...,Reclamação,2021-10-27,2022-10-25,62,PROJETO CONTEMPLA EXTENSÃO E DESATIVAÇÃO DA R...,NaN,NaN,-22.013691,-56.498904,5002100


In [ ]:
# Realizar o merge com os nomes de coluna corretos
SIAGO_C622 = SIAGO_C622.merge(
    GS[['Obra', 'Data Limite']],
    left_on='num_obra',         # Nome da coluna no SS_EED
    right_on='Obra',   # Nome da coluna no GS
    how='left',
    suffixes=('', '_Obra')  # Adiciona sufixos para evitar duplicatas
)
# Excluindo uma coluna após o merge
SIAGO_C622 = SIAGO_C622.drop('Obra', axis=1)


In [ ]:
SIAGO_C622.head(5)

,status_dsc_status,nome,sigla,val_material,dscsit_mat,cod_empreiteira,rzo_social,num_obra,tip_envolvido,cod_envolvido,...,vlr_uc_rea,vlr_com_prv,vlr_com_rea,vlr_ca_prv,vlr_ca_rea,uc_com_controlado,ind_nivel_corrosao,Codigo Material,Simplificado,Data Limite
0,ENCERRAMENTO:ATUALIZAÇÃO DO CADASTRO,NELSON DE FREITAS LUCIO,EN404102,"1082,27",NORMAL,64709,LIGHTSUL INSTALACOES ELETRICAS LTDA,101806728,C,NaN,...,0,0,0,"25019,87","30073,51",S,N,NaN,NaN,2018-10-26
1,ENCERRAMENTO:ATUALIZAÇÃO DO CADASTRO,NELSON DE FREITAS LUCIO,EN404102,"2,28",DESPADRONIZADO,64709,LIGHTSUL INSTALACOES ELETRICAS LTDA,101806728,C,NaN,...,0,0,0,"25019,87","30073,51",S,N,NaN,NaN,2018-10-26
2,ENCERRAMENTO:ATUALIZAÇÃO DO CADASTRO,NELSON DE FREITAS LUCIO,EN404102,"8,7806",NORMAL,64709,LIGHTSUL INSTALACOES ELETRICAS LTDA,101806728,C,NaN,...,0,0,0,"25019,87","30073,51",N,N,NaN,NaN,2018-10-26
3,ENCERRAMENTO:ATUALIZAÇÃO DO CADASTRO,NELSON DE FREITAS LUCIO,EN404102,"223,6601",DESPADRONIZADO,64709,LIGHTSUL INSTALACOES ELETRICAS LTDA,101806728,C,NaN,...,0,0,0,"25019,87","30073,51",N,N,NaN,NaN,2018-10-26
4,ENCERRAMENTO:ATUALIZAÇÃO DO CADASTRO,NELSON DE FREITAS LUCIO,EN404102,"43,12",DESPADRONIZADO,64709,LIGHTSUL INSTALACOES ELETRICAS LTDA,101806728,C,NaN,...,0,0,0,"25019,87","30073,51",N,N,NaN,NaN,2018-10-26


In [ ]:
# Filtrar onde não é NaN e não é string vazia
SIAGO_C622 = SIAGO_C622[SIAGO_C622['Codigo Material'].notna() & (SIAGO_C622['Codigo Material'] != '')]

# Converter a coluna 'Codigo Material' para inteiro
SIAGO_C622['Codigo Material'] = SIAGO_C622['Codigo Material'].astype(float).astype(int)

display(SIAGO_C622['Codigo Material'])


2618        5045
2619        5046
2687      610808
2688      629431
2689      634610
           ...  
460475    690787
460488    690001
461541      5046
468744    641331
468985    641331
Name: Codigo Material, Length: 2712, dtype: int64

In [ ]:

# Supondo que você já tenha o DataFrame SS_EED carregado e manipulado

# Defina o caminho e o nome do arquivo Excel de saída
caminho_arquivo = 'SIAGO_C622.xlsx'

# Exportar o DataFrame para um arquivo Excel
SIAGO_C622.to_excel(caminho_arquivo, index=False, sheet_name='SIAGO_C622')

print(f"Arquivo Excel salvo em: {caminho_arquivo}")


Arquivo Excel salvo em: SIAGO_C622.xlsx


In [ ]:
RESUMO_EQ = RESUMO_EQ.merge(
    SIAGO_C611[['num_obra', 'tipo_servico', 'status_atual', 'dscgrp_obra','descricao','num_projeto_sigco','dsc_prazo_conclusao','data_hora']],
    left_on='OBRA',
    right_on='num_obra',
    how='left'
)

In [ ]:
# Função para mapear as espécies
def map_Equipamento(Equipamento):
    if Equipamento == "Banco de capacitor":
        return "BD"
    elif Equipamento == "":
        return "Verificar"
    elif Equipamento == "Religador Monofásico":
        return "RDM"
    elif Equipamento == "Regulador de Tensão":
        return "RT"
    elif Equipamento == "Religador Trifásico":
        return "RD"
    else:
        return ""

# Aplicar a função ao DataFrame e criar uma nova coluna
RESUMO_EQ['Equipamento Mapeado'] = RESUMO_EQ['Equipamento'].apply(map_Equipamento)


In [ ]:
# Função para determinar se o parecer Telecom é necessário
def verificar_necessidade_telecom(equipamento):
    if equipamento == "Banco de capacitor":
        return "Não Necessita Parecer Telecom"
    else:
        return "Necessita Parecer Telecom"

# Aplicar a função e criar uma nova coluna para o parecer Telecom
RESUMO_EQ['Necessidade Parecer Telecom'] = RESUMO_EQ['Equipamento'].apply(verificar_necessidade_telecom)

# Visualizar o DataFrame atualizado
display(RESUMO_EQ[['Equipamento', 'Parecer Telecom ', 'SS_Telecom', 'Necessidade Parecer Telecom']].head())

,Equipamento,Parecer Telecom,SS_Telecom,Necessidade Parecer Telecom
0,Religador Trifásico,Solicitado,EMS-TEL-EST 00115/2024,Necessita Parecer Telecom
1,Banco de capacitor,OK,N.A,Não Necessita Parecer Telecom
2,Banco de capacitor,OK,N.A,Não Necessita Parecer Telecom
3,Banco de capacitor,OK,OK,Não Necessita Parecer Telecom
4,Banco de capacitor,OK,OK,Não Necessita Parecer Telecom


In [ ]:
# Validar itens distintos na coluna "Parecer Telecom"
distinct_values = RESUMO_EQ['Parecer Telecom '].unique()
display(distinct_values)

array(['Solicitado', 'OK', nan, 'N.A', 'Viável', 'Solicitar',
       'Teste de Propagação', 'Viável com ressalva', 'Cancelada',
       'Inviável'], dtype=object)

In [ ]:
# Realizar o merge com os nomes de coluna corretos
RESUMO_EQ = RESUMO_EQ.merge(
    GS[['Obra', 'Empreiteira', 'Status_GS', 'Inicio Físico']],
    left_on='OBRA',         # Nome da coluna no SS_EED
    right_on='Obra',   # Nome da coluna no GS
    how='left',
    suffixes=('', '_Obra')  # Adiciona sufixos para evitar duplicatas
)

In [ ]:
RESUMO_EQ = RESUMO_EQ.merge(
    BASE_ADMs[['MODELO', 'Nome', 'Master', 'Número Operativo']],
    left_on='Número Equipamento',
    right_on='Número Operativo',
    how='left'
)


In [ ]:

# Converter a coluna 'Coordenada X UTM' para numérico
RESUMO_EQ['Coordenada X UTM'] = pd.to_numeric(RESUMO_EQ['Coordenada X UTM'], errors='coerce')
# Converter a coluna 'Coordenada X UTM' para numérico
RESUMO_EQ['Coordenada Y UTM'] = pd.to_numeric(RESUMO_EQ['Coordenada Y UTM'], errors='coerce')



In [ ]:
print(RESUMO_EQ.columns)

Index(['Status', 'Ano', 'ID', 'SIGCO', 'Equipamento', 'Referência',
       'Tipo de atuação', 'Configuração (NF/NA)', 'Equipamento operando?',
       'Fase', 'Qntd Final Fases', 'kVAr', 'Capacidade Nominal (A)', 'Origem',
       'Região', 'Supervisão', 'Localidade', 'Conjunto', 'Alimentador',
       'Tensão', 'Zona', 'Zona UTM', 'Coordenada X UTM', 'Coordenada Y UTM',
       'Parecer Telecom ', 'SS_Telecom', 'DATA DE SOLICITAÇÃO ',
       'DATA DE RETORNO Telecom', 'MASTER', 'Repetidora',
       'Nível de sinal de RX ', 'Altura da antena', 'Antena / Modelo e Ganho',
       'Rádio / Fabricante e Modelo', 'Atenuador de RF / Modelo',
       'Necessidade de Super Poste', 'OBRA', 'Número Equipamento', 'Cod_Ativo',
       'SS', 'OS', 'SAC CADASTRO', 'Dt Solicitação SAC', 'Dt Atendimento SAC',
       'Zona UTM.1', 'Coordenada X UTM.1', 'Coordenada Y UTM.1',
       'Nome do equipamento', 'Linha', 'FABRICANTE SUCATA', 'FABRICANTE NOVO',
       'EMPREITEIRA', 'Equipamento enviado?', 'EMD SICEC',

In [ ]:
# Remover espaços em branco dos nomes das colunas
SS_EED.columns = SS_EED.columns.str.strip()
print(SS_EED.columns)

Index(['Código da Solicitação', 'Ano', 'Descrição', 'DataOcorrência',
       'Abrangencia', 'Instalação', 'Trecho', 'Localização', 'Complemento',
       'Sigla Orgao', 'SolicNome', 'DescPrioridade', 'SS-Estados',
       'Data/Hora Limite', 'Ativos', 'Qtd_Clientes', 'Espécies', 'SS-Tipos',
       'Término', 'OSNumero', 'OS-Estados', 'PlanDataInicio', 'PlanDataFinal',
       'DescOrigem', 'Condição Operativa COI', 'Descrição do Defeito', 'Lida',
       'Processo', 'DEPARTCODE', 'UF', 'Bairro', 'Logradouro', 'OS SIATT',
       'SS PRINCIPAL', 'SOMA', 'Coordenada Longitude(X)',
       'Coordenada Latitude(Y)', 'Equipamento Operando?', 'REGIONAL',
       'CÓDIGO REGIONAL', 'ESTRUTURA', 'EST PRIMÁRIA', 'EST SECUNDÁRIA',
       'NOTA', 'TIPO NOTA', 'ORDEM', 'LOCALIDADE', 'CODIGO CLIENTE',
       'CARACTERÍSTICA REGIONAL', 'Equipe de atuação', 'DEPARTAMENTO',
       'TIPO EED', 'Número Operativo', 'CODIGO_INSTALACAO', 'FASES', 'TRECHO',
       'SUPERVISÃO', 'NOME DO RELIGADOR', 'LZ_TIPO', 'EST

In [ ]:
RESUMO_EQ.columns = RESUMO_EQ.columns.str.strip()


In [ ]:
RESUMO_EQ = RESUMO_EQ.merge(
    SS_EED[['SS-Estados', 'Término', 'Código da Solicitação','DataOcorrência']],
    left_on='SS_Telecom',
    right_on='Código da Solicitação',
    how='left'
)
# Renomear a coluna 'Código da Solicitação' para 'CodigoSolicitacao'
RESUMO_EQ.rename(columns={'Código da Solicitação': 'SSs_Telecom'}, inplace=True)
RESUMO_EQ.rename(columns={'SS-Estados': 'SS-Estados_Telecom'}, inplace=True)
RESUMO_EQ.rename(columns={'Término': 'Término_SS_Telecom'}, inplace=True)
RESUMO_EQ.rename(columns={'DataOcorrência': 'DataOcorrência_SS_Telecom'}, inplace=True)

In [ ]:

RESUMO_EQ = RESUMO_EQ.merge(
    SS_EED[['SS-Estados', 'Término', 'Código da Solicitação','DataOcorrência']],
    left_on='OAP / OAV',
    right_on='Código da Solicitação',
    how='left'
)
# Renomear a coluna 'Código da Solicitação' para 'CodigoSolicitacao'
RESUMO_EQ.rename(columns={'Código da Solicitação': 'SS_OAP / OAV'}, inplace=True)
RESUMO_EQ.rename(columns={'SS-Estados': 'SS-Estados_OAP / OAV'}, inplace=True)
RESUMO_EQ.rename(columns={'Término': 'Término_SS_OAP / OAV'}, inplace=True)
RESUMO_EQ.rename(columns={'DataOcorrência': 'DataOcorrência_SS_OAP / OAV'}, inplace=True)

In [ ]:
RESUMO_EQ = RESUMO_EQ.merge(
    SS_EED[['SS-Estados', 'Término', 'Código da Solicitação','DataOcorrência']],
    left_on='Tela do SCADA',
    right_on='Código da Solicitação',
    how='left'
)
# Renomear a coluna 'Código da Solicitação' para 'CodigoSolicitacao'
RESUMO_EQ.rename(columns={'Código da Solicitação': 'SS_Tela do SCADA'}, inplace=True)
RESUMO_EQ.rename(columns={'SS-Estados': 'SS-Estados_Tela do SCADA'}, inplace=True)
RESUMO_EQ.rename(columns={'Término': 'Término_SS_Tela do SCADA'}, inplace=True)
RESUMO_EQ.rename(columns={'DataOcorrência': 'DataOcorrência_SS_Tela do SCADA'}, inplace=True)

In [ ]:
RESUMO_EQ = RESUMO_EQ.merge(
    SS_EED[['SS-Estados', 'Término', 'Código da Solicitação','DataOcorrência']],
    left_on='SS - DEMT/DEOP',
    right_on='Código da Solicitação',
    how='left'
)
# Renomear a coluna 'Código da Solicitação' para 'CodigoSolicitacao'
RESUMO_EQ.rename(columns={'Código da Solicitação': 'SS_SS - DEMT/DEOP'}, inplace=True)
RESUMO_EQ.rename(columns={'SS-Estados': 'SS-Estados_SS - DEMT/DEOP'}, inplace=True)
RESUMO_EQ.rename(columns={'Término': 'Término_SS_SS - DEMT/DEOP'}, inplace=True)
RESUMO_EQ.rename(columns={'DataOcorrência': 'DataOcorrência_SS_SS - DEMT/DEOP'}, inplace=True)

In [ ]:
# Implementação da lógica do IF (DAX) em Python
RESUMO_EQ['Teste_Troca_Marca'] = RESUMO_EQ.apply(
    lambda row: 'Mesma Marca' if row['FABRICANTE NOVO'] == row['FABRICANTE SUCATA'] else 'Diferente', axis=1
)

In [ ]:
RESUMO_EQ['Necessita_comissionar'] = RESUMO_EQ.apply(
    lambda row: 'COMISSIONAR' if row['Tipo de atuação'] in ["Instalação", "Realocação", "Substituição", "Instalação/Desistalação"] else 'Outros', axis=1
)


In [ ]:
# Implementação da lógica do IF (DAX) em Python
RESUMO_EQ['Solicitar_Protecao'] = RESUMO_EQ.apply(
    lambda row: 'SOLICITAR' if row['Necessita_comissionar'] == 'Comissionar' 
                                and pd.isna(row['OAP / OAV']) 
                                and row['Teste_Troca_Marca'] == 'Diferente' 
                            else 'SOLICITAR', axis=1
)

In [ ]:
print(RESUMO_EQ['Solicitar_Protecao'])

0       SOLICITAR
1       SOLICITAR
2       SOLICITAR
3       SOLICITAR
4       SOLICITAR
          ...    
3400    SOLICITAR
3401    SOLICITAR
3402    SOLICITAR
3403    SOLICITAR
3404    SOLICITAR
Name: Solicitar_Protecao, Length: 3405, dtype: object


In [ ]:
print(RESUMO_EQ)

        Status   Ano  ID  SIGCO          Equipamento  Referência  \
0     LIBERADA  2022  25  45128  Religador Trifásico   FU(21864)   
1     LIBERADA  2021  MN   8822   Banco de capacitor   BC(10293)   
2     LIBERADA  2021  MN   8822   Banco de capacitor   BC(10293)   
3     LIBERADA  2022  11  44768   Banco de capacitor  CF(742192)   
4     LIBERADA  2022  11  44768   Banco de capacitor  CF(742192)   
...        ...   ...  ..    ...                  ...         ...   
3400  LIBERADA  2024  NC  54674  Regulador de Tensão   FU(25634)   
3401  LIBERADA  2024  NC  54674  Regulador de Tensão   FU(25634)   
3402  LIBERADA  2024  NC  11828  Regulador de Tensão   FU(30037)   
3403  LIBERADA  2024  NC  11828  Religador Trifásico   FU(21740)   
3404  LIBERADA  2024  NC  11826  Religador Trifásico   FU(11695)   

     Tipo de atuação Configuração (NF/NA) Equipamento operando? Fase  ...  \
0         Instalação                  NaN                   Não  ABC  ...   
1        Aterramento        N

In [ ]:

# Caminho da pasta de origem
# origem = r'\\fs-bi\EMS\DTEC\DCMD\DW_DCMD\SICEC'

# Caminho da pasta de destino
# destino = r'C:\Users\EN405081\Projetos Python\BI_EQUIPAMENTOS'

# Nomes dos arquivos que você quer transferir
# arquivos_para_transferir = [
  #   'SICEC_C111_Estoque_Materiais.txt', 
    # 'SICEC_C211_Movimentacao_Materiais_Definitivo_2024.txt',
    # 'SICEC_C212_Movimentacao_Materiais_Temporario_2024.txt'
# ]

# Copia cada arquivo para a pasta de destino, substituindo o existente
# for arquivo in arquivos_para_transferir:
  #   caminho_completo_origem = os.path.join(origem, arquivo)
    # caminho_completo_destino = os.path.join(destino, arquivo)
    
    # Copiar o arquivo, incluindo metadados (e substituindo o existente, se houver)
  #   shutil.copy2(caminho_completo_origem, caminho_completo_destino)

# print("Transferência concluída!")



In [ ]:
SICEC_C111_Estoque_Materiais = pd.read_csv(
    r'C:\Users\EN405081\Projetos Python\BI_EQUIPAMENTOS\SICEC\SICEC_C111_Estoque_Materiais.txt',
    sep='\t',  # or another delimiter, e.g., ',' if it's a CSV
    encoding='latin1',
    on_bad_lines='skip',  # Skip bad lines instead of 'error_bad_lines'
    low_memory=False
)

print(SICEC_C111_Estoque_Materiais.columns)

Index(['ï»¿material_codmat', 'material_dscmat', 'material_codund_mda_mat',
       'material_vlrult_cot', 'material_datault_cot', 'deposito_codreg_dep',
       'deposito_qtdesq_dep', 'deposito_qtdesq_dep_1', 'deposito_qtdesq_dep_2',
       'deposito_qtdesq_dep_3'],
      dtype='object')


In [ ]:
SICEC_C211_Movimentacao_Materiais_Definitivo_2024 = pd.read_csv(
    r'C:\Users\EN405081\Projetos Python\BI_EQUIPAMENTOS\SICEC\SICEC_C211_Movimentacao_Materiais_Definitivo_2024.txt',
    sep='\t',  # or another delimiter, e.g., ',' if it's a CSV
    encoding='latin1',
    on_bad_lines='skip',  # Skip bad lines instead of 'error_bad_lines'
    low_memory=False
)
print(SICEC_C211_Movimentacao_Materiais_Definitivo_2024.columns)

Index(['ï»¿nummov', 'orgmov', 'numdoc_mov', 'datamov', 'codarea_mov',
       'codcta_mov', 'codsct_mov', 'anoeph_mov', 'meseph_mov', 'depdest',
       'sigla_mov_mat', 'numitem', 'codamx_mov', 'codmat_mov', 'qtdmov',
       'vlrtot_mov', 'codreg_mov', 'codund_mda_mat', 'dscmat', 'dth_inclusao',
       'nummov_tmp', 'estoque_regulador', 'codemp_req', 'ind_material_orcado',
       'numptl_mov_med', 'cod_natureza_lancamento', 'indctz_mov',
       'ind_consumo_material', 'codcta_obra', 'codsct_obra', 'indorc_dsp_inv',
       'num_lista_movimentacao', 'codrsp_rec', 'nomsis_ori_slc', 'codcls_mat',
       'uf_origem', 'uf_destino', 'num_plano_sienf', 'data_solicitacao',
       'data_necessidade', 'data_separacao', 'dsc_romaneios', 'tip_obra',
       'cod_prazo_conclusao', 'num_obra', 'cod_almoxarifado_operacional',
       'num_inventario', 'dsc_mensagem_processamento',
       'num_protocolo_equipamento'],
      dtype='object')


In [ ]:
SICEC_C212_Movimentacao_Materiais_Temporario_2024 = pd.read_csv(
    r'C:\Users\EN405081\Projetos Python\BI_EQUIPAMENTOS\SICEC\SICEC_C212_Movimentacao_Materiais_Temporario_2024.txt',
    sep='\t',  # or another delimiter, e.g., ',' if it's a CSV
    encoding='latin1',
    on_bad_lines='skip',  # Skip bad lines instead of 'error_bad_lines'
    low_memory=False
)

print(SICEC_C212_Movimentacao_Materiais_Temporario_2024.columns)

Index(['ï»¿nummov', 'orgmov', 'numdoc_mov', 'datamov', 'codarea_mov',
       'codcta_mov', 'codsct_mov', 'anoeph_mov', 'meseph_mov', 'depdest',
       'sigla_mov_mat', 'numitem', 'codamx_mov', 'codmat_mov', 'qtdmov',
       'qtdatd', 'codreg_mov', 'codund_mda_mat', 'dscmat', 'indsit',
       'codemp_req', 'dscobs_slc', 'nomsis_ori_slc', 'qtdrsv', 'qtdcan',
       'cod_natureza_lancamento', 'codrsp_rec', 'codcls_mat', 'uf_origem',
       'uf_destino', 'datncs_etg', 'datpvs_etg', 'qtdpend', 'nomecdr'],
      dtype='object')


In [ ]:
RESUMO_EQ=RESUMO_EQ.merge(
SICEC_C212_Movimentacao_Materiais_Temporario_2024[['ï»¿nummov','datamov','depdest']],
 left_on='EMD SICEC',
    right_on='ï»¿nummov',
    how='left'

)
RESUMO_EQ.rename(columns={'ï»¿nummov': 'EMD'}, inplace=True)
RESUMO_EQ.rename(columns={'datamov': 'Data Solicitação'}, inplace=True)
RESUMO_EQ.rename(columns={'depdest': 'Deposito destino'}, inplace=True)

In [ ]:


# Supondo que você já tenha o DataFrame SS_EED carregado e manipulado

# Defina o caminho e o nome do arquivo Excel de saída
caminho_arquivo = 'Resumoeq.xlsx'

# Exportar o DataFrame para um arquivo Excel
RESUMO_EQ.to_excel(caminho_arquivo, index=False, sheet_name='RESUMO_EQ')

print(f"Arquivo Excel salvo em: {caminho_arquivo}")


Arquivo Excel salvo em: Resumoeq.xlsx


In [ ]:
SIGPO_C611_Dados_de_Obras_Construção = pd.read_csv(
    r'SIGPO_C611_Dados_de_Obras_Construção.txt',
    sep='\t',  # or another delimiter, e.g., ',' if it's a CSV
    encoding='latin1',
    on_bad_lines='skip',  # Skip bad lines instead of 'error_bad_lines'
    low_memory=False
)

In [ ]:
# Filtrar a coluna 'cod_regional' para obter apenas as linhas com valor 10
SIGPO_C611_Dados_de_Obras_Construção = SIGPO_C611_Dados_de_Obras_Construção[SIGPO_C611_Dados_de_Obras_Construção['mt_especifico'] == "SIM"]

display(SIGPO_C611_Dados_de_Obras_Construção)



,cod_regional,num_obra,numobra_asc,numobra_dsc,cod_area,desc_tipo_envolvido,desc_classificacao_recurso,descricao,desc_tipo_servico,tip_caracteristica,...,obra_dia_suspensao,asbuilt_concluido,indept_pcp,codloc,nomeloc,numos_orig,dth_prim_aprov_orc_clien,suspensao_os_projeto,ind_geracao_distribuida,Local_Base
12,60,602300569,NaN,NaN,192301000,CLIENTE,ORDEM DE IMOBILIZAÇÃO,CIA x 3º - Carga > 50kW - ERD,CONEXÃO REDE p-LIG NOVA GRUPO A,NaN,...,0,N,S,3357.0,PARANAIBA,NaN,31/07/2024 16:14:47,0,S,611_2024
19,60,602400937,NaN,NaN,190500310,CLIENTE,ORDEM DE IMOBILIZAÇÃO,CIA x TERCEIROS VEGETATIVO,INSTALAÇÃO DE TRAFO,NaN,...,0,N,S,3308.0,APARECIDA DO TABOADO,NaN,17/10/2024 14:45:11,0,N,611_2024
21,20,202301250,NaN,NaN,192501000,CLIENTE,ORDEM DE IMOBILIZAÇÃO,CIA x 3º - Carga > 50kW - ERD,CONEXÃO REDE p-LIG NOVA GRUPO A,NaN,...,272,N,S,3339.0,ITAPORA,125399475.0,03/11/2023 15:31:18,0,N,611_2024
30,70,702400533,NaN,NaN,190500310,CLIENTE,NaN,CIA x 3º Carga < 50kW - ERD,REFORÇO REDE sem PE p/AUM CARGA- GRUPO B,NaN,...,0,N,S,3365.0,RIO VERDE,297583120.0,NaN,0,S,611_2024
37,40,402400086,NaN,NaN,192501000,EMPRESA,ORDEM DE IMOBILIZAÇÃO,PRÓPRIA-CONSTRUÇÃO ALIMENTADOR,INSTALAÇÃO DE RELIGADOR,NaN,...,0,N,S,3303.0,AMAMBAI,NaN,NaN,0,N,611_2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218681,13,131800210,NaN,NaN,190900000,EMPRESA,ORDEM DE DESATIVAÇÃO,PERDAS: FISCALIZAÇÃO GRUPO B,OUTROS,NaN,...,0,N,NaN,3320.0,CAMPO GRANDE,NaN,NaN,0,N,611_full
218841,20,201806124,NaN,NaN,190500000,CLIENTE,ORDEM DE SERVIÇO,CIA x TERCEIROS VEGETATIVO,EXTENSÃO DE REDE,NaN,...,0,N,S,3332.0,DOURADOS,NaN,11/04/2017 00:00:02,0,N,611_full
219147,10,101812361,NaN,NaN,192301000,CLIENTE,ORDEM DE IMOBILIZAÇÃO,CIA x TERCEIROS VEGETATIVO,CONSTR. DE RAMAL RURAL,NaN,...,0,N,NaN,3366.0,ROCHEDO,NaN,NaN,0,N,611_full
219201,20,201806126,NaN,NaN,190500000,CLIENTE,ORDEM DE SERVIÇO,CIA x TERCEIROS VEGETATIVO,Padrão de Entrada e/ou Kit Interno,NaN,...,0,N,S,3332.0,DOURADOS,NaN,22/02/2017 00:00:00,0,N,611_full


In [ ]:
SIGPO_C611_Dados_de_Obras_Construção=SIGPO_C611_Dados_de_Obras_Construção.merge(
RESUMO_EQ[['OBRA','Número Equipamento']],
 left_on='num_obra',
    right_on='OBRA',
    how='left'

)

In [ ]:
# Exportar o DataFrame para um arquivo TXT
caminho_arquivo_txt = 'SIGPO_C611_Dados_de_Obras.txt'
SIGPO_C611_Dados_de_Obras_Construção.to_csv(caminho_arquivo_txt, sep='\t', index=False, encoding='utf-8')

print(f"Arquivo TXT salvo em: {caminho_arquivo_txt}")


Arquivo TXT salvo em: SIGPO_C611_Dados_de_Obras.txt


In [ ]:

# Função para remover caracteres ilegais
def remover_caracteres_ilegais(texto):
    if isinstance(texto, str):
        # Substitui caracteres ilegais (ASCII 0-31, exceto tab, newline, e carriage return)
        return ''.join(char for char in texto if ord(char) >= 32 or char in ['\t', '\n', '\r'])
    return texto

# Aplicar a função para todas as colunas do DataFrame
SIGPO_C611_Dados_de_Obras_Construção = SIGPO_C611_Dados_de_Obras_Construção.applymap(remover_caracteres_ilegais)

# Exportar o DataFrame para um arquivo Excel após limpar os caracteres ilegais
caminho_arquivo = 'SIGPO_C611.xlsx'
SIGPO_C611_Dados_de_Obras_Construção.to_excel(caminho_arquivo, index=False, sheet_name='SIGPO_C611')

print(f"Arquivo Excel salvo em: {caminho_arquivo}")


C:\Users\EN405081\AppData\Local\Temp\ipykernel_7504\3918888698.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  SIGPO_C611_Dados_de_Obras_Construção = SIGPO_C611_Dados_de_Obras_Construção.applymap(remover_caracteres_ilegais)


Arquivo Excel salvo em: SIGPO_C611.xlsx


In [ ]:
# Agrupar por 'EQUIPAMENTO' e contar os valores únicos de 'CODIGO_INSTALACAO'
qd_SS_DCMD = SS_EED.groupby('DEPARTAMENTO')['Código da Solicitação'].nunique().reset_index()

# Renomear a coluna de contagem para algo mais descritivo
qd_SS_DCMD.rename(columns={'Código da Solicitação': 'Quantidade_SS'}, inplace=True)

print(qd_SS_DCMD)

             DEPARTAMENTO  Quantidade_SS
0                                  15262
1                    ASPO            130
2                    DCMD           1933
3                    DEMT           6721
4                    DEOP           2151
5   ENGESELT-LEVANTAMENTO              2
6             OBRA P/2025             19
7                PROTEÇÃO            481
8            PRÉ-OPERAÇÃO            190
9                    SPMA            208
10                TELECON            396


In [ ]:
SS_EED.head(5)

,Código da Solicitação,Ano,Descrição,DataOcorrência,Abrangencia,Instalação,Trecho,Localização,Complemento,Sigla Orgao,...,DataCriação,Obra,Empreiteira,Status_GS,Inicio Físico,MODELO,Nome,Master,num_obra,status_atual
0,EMS-RD-NO 00048/2020,2020,troca de trafo quemado,11/08/2020 16:09:18,23572,IVI03,NaN,108227,ENTT108227,COI,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,EMS-RD-CG 01708/2018,2018,SINALIZANDO BATERIA CC DEFEITO / ALIMENTAÇÃO C...,09/10/2018 01:25:30,41398,SIA51,2,453RL006812819,453RL006812819,COI,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,EMS-SE-SU 00046/2019,2019,RELE 50/51 SINALIZA DEFEITO PARA O SSC/COS.,27/01/2019 09:54:32,BAY-65TF-02,SPOV,NaN,65TF-02,SPOV-65TF-02,COI,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,EMS-DEMT 00097/2019,2019,Saneamento do sistema (OS's 2018-2020).,10/01/2019 17:19:22,NaN,SCMS,NaN,CMS,SCMS,MANUT,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4,EMS-OP-CG 00001/2019,2019,PROGRAMAR TROCA DE POSTE DT 10/150 PARA DT 11/...,04/01/2019 08:50:51,NaN,SIA03,NaN,NaN,235426055,COI,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Exibir algumas amostras da coluna 'DataOcorrência'
print(SS_EED['DataOcorrência'].sample(10))


7206     05/06/2021 06:36:31
25340    07/11/2023 12:49:00
8355     11/02/2021 16:00:07
4783     24/08/2020 11:25:20
21560    07/03/2023 10:23:34
3206     29/05/2020 17:15:20
3347     24/03/2020 15:53:59
23798    16/10/2024 17:12:25
21462    02/10/2023 08:16:33
11695    20/07/2021 09:51:38
Name: DataOcorrência, dtype: object


In [ ]:


# Função para tentar converter a data
def parse_date(date_str):
    for fmt in ('%d/%m/%Y %H:%M:%S', '%Y-%m-%d %H:%M:%S', '%d-%m-%Y %H:%M:%S'):
        try:
            return pd.to_datetime(date_str, format=fmt, errors='raise')
        except ValueError:
            continue
    return pd.NaT  # Retorna NaT se nenhum formato funcionar

# Aplicar a função na coluna de datas
SS_EED['DataOcorrência'] = SS_EED['DataOcorrência'].apply(parse_date)

# Verifique se há valores NaT e trate-os conforme necessário
print(SS_EED['DataOcorrência'].isna().sum())  # Conta quantas datas não puderam ser convertidas



5


In [ ]:
# Excluir linhas com datas inválidas
SS_EED = SS_EED.dropna(subset=['DataOcorrência'])


In [ ]:
# Exibir algumas amostras da coluna 'DataOcorrência'
print(SS_EED['DataOcorrência'].sample(10))

2538    2020-02-17 18:06:19
158     2020-02-14 21:26:50
15353   2022-08-04 14:14:56
12913   2022-05-16 07:15:12
25023   2024-10-16 12:34:43
6604    2020-07-15 16:44:00
3537    2020-06-25 11:45:50
21349   2023-09-05 10:58:37
21173   2023-09-05 17:26:56
1071    2020-02-17 21:09:46
Name: DataOcorrência, dtype: datetime64[ns]


In [ ]:
# Filtros para considerar apenas as SS atendidas e pendentes
filtros = SS_EED['SS-Estados'].isin(['SS ATENDIDA', 'SS PENDENTE'])


# Certifique-se de que a coluna de data está no formato datetime
SS_EED['DataOcorrência'] = pd.to_datetime(SS_EED['DataOcorrência'], dayfirst=True)

# Crie colunas separadas para o ano e o mês
SS_EED['ANO'] = SS_EED['DataOcorrência'].dt.year
SS_EED['MES'] = SS_EED['DataOcorrência'].dt.month

# Agora, agrupe por 'DEPARTAMENTO', 'ANO' e 'MES' e conte os valores únicos de 'Código da Solicitação'
qd_SS_DEPARTAMETO = SS_EED.groupby(['DEPARTAMENTO', 'ANO', 'MES'])['Código da Solicitação'].nunique().reset_index()

# Renomeie as colunas para facilitar a leitura
qd_SS_DEPARTAMETO.columns = ['DEPARTAMENTO', 'ANO', 'MES', 'QUANTIDADE_SS']


In [ ]:
# Função para tentar converter a data
def parse_date(date_str):
    for fmt in ('%d/%m/%Y %H:%M:%S', '%Y-%m-%d %H:%M:%S', '%d-%m-%Y %H:%M:%S'):
        try:
            return pd.to_datetime(date_str, format=fmt, errors='raise')
        except ValueError:
            continue
    return pd.NaT  # Retorna NaT se nenhum formato funcionar

# Aplicar a função na coluna de datas
SS_EED['Término'] = SS_EED['Término'].apply(parse_date)

# Verifique se há valores NaT e trate-os conforme necessário
print(SS_EED['Término'].isna().sum())  # Conta quantas datas não puderam ser convertidas



7200


In [ ]:
# Certifique-se de que a coluna de data está no formato datetime
SS_EED['Término'] = pd.to_datetime(SS_EED['Término'], dayfirst=True)

# Crie colunas separadas para o ano e o mês
SS_EED['ANO'] = SS_EED['Término'].dt.year
SS_EED['MES'] = SS_EED['Término'].dt.month

# Agora, agrupe por 'DEPARTAMENTO', 'ANO' e 'MES' e conte os valores únicos de 'Código da Solicitação'
qd_SS_ATENDIDA = SS_EED.groupby(['DEPARTAMENTO', 'ANO', 'MES'])['Código da Solicitação'].nunique().reset_index()

# Renomeie as colunas para facilitar a leitura
qd_SS_ATENDIDA.columns = ['DEPARTAMENTO', 'ANO', 'MES', 'QUANTIDADE_SAIU']

In [ ]:
display(qd_SS_ATENDIDA)
display(qd_SS_DEPARTAMETO)

,DEPARTAMENTO,ANO,MES,QUANTIDADE_SAIU
0,,2019.0,6.0,1
1,,2020.0,1.0,187
2,,2020.0,2.0,195
3,,2020.0,3.0,346
4,,2020.0,4.0,297
...,...,...,...,...
341,TELECON,2024.0,5.0,14
342,TELECON,2024.0,7.0,16
343,TELECON,2024.0,8.0,1
344,TELECON,2024.0,10.0,28


,DEPARTAMENTO,ANO,MES,QUANTIDADE_SS
0,,2018,8,2
1,,2018,10,1
2,,2019,1,5
3,,2019,3,2
4,,2019,4,1
...,...,...,...,...
382,TELECON,2024,7,7
383,TELECON,2024,8,3
384,TELECON,2024,9,12
385,TELECON,2024,10,18


In [ ]:


# Realizar a junção
resultado = pd.merge(qd_SS_DEPARTAMETO, qd_SS_ATENDIDA, on=['DEPARTAMENTO', 'ANO', 'MES'], how='outer')

# Preencher NaNs com 0 nas colunas de quantidade
resultado['QUANTIDADE_SS'].fillna(0, inplace=True)
resultado['QUANTIDADE_SAIU'].fillna(0, inplace=True)

# Calcular a diferença
resultado['DIFERENCA'] = resultado['QUANTIDADE_SS'] - resultado['QUANTIDADE_SAIU']

# Renomear colunas, se necessário, para facilitar a leitura
resultado = resultado[['DEPARTAMENTO', 'ANO', 'MES', 'QUANTIDADE_SS', 'QUANTIDADE_SAIU', 'DIFERENCA']]

# Exibir o resultado final
resultado.head(30)

C:\Users\EN405081\AppData\Local\Temp\ipykernel_7504\1251347344.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  resultado['QUANTIDADE_SS'].fillna(0, inplace=True)
C:\Users\EN405081\AppData\Local\Temp\ipykernel_7504\1251347344.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

,DEPARTAMENTO,ANO,MES,QUANTIDADE_SS,QUANTIDADE_SAIU,DIFERENCA
0,,2018.0,8.0,2.0,0.0,2.0
1,,2018.0,10.0,1.0,0.0,1.0
2,,2019.0,1.0,5.0,0.0,5.0
3,,2019.0,3.0,2.0,0.0,2.0
4,,2019.0,4.0,1.0,0.0,1.0
5,,2019.0,6.0,1.0,1.0,0.0
6,,2019.0,8.0,3.0,0.0,3.0
7,,2019.0,9.0,1.0,0.0,1.0
8,,2019.0,10.0,7.0,0.0,7.0
9,,2019.0,11.0,5.0,0.0,5.0


In [ ]:

# Calcular a diferença inicial
resultado['DIFERENCA'] = resultado['QUANTIDADE_SS'] - resultado['QUANTIDADE_SAIU']

# Ordenar os dados para garantir a sequência correta de ANO e MES dentro de cada DEPARTAMENTO
resultado = resultado.sort_values(by=['DEPARTAMENTO', 'ANO', 'MES'])

# Acumular as pendências para o mês seguinte dentro de cada DEPARTAMENTO
resultado['DIFERENCA_ACUMULADA'] = resultado.groupby(['DEPARTAMENTO', 'ANO'])['DIFERENCA'].cumsum()

# Exibir o resultado final
print(resultado[['DEPARTAMENTO', 'ANO', 'MES', 'QUANTIDADE_SS', 'QUANTIDADE_SAIU', 'DIFERENCA', 'DIFERENCA_ACUMULADA']])

    DEPARTAMENTO     ANO   MES  QUANTIDADE_SS  QUANTIDADE_SAIU  DIFERENCA  \
0                 2018.0   8.0            2.0              0.0        2.0   
1                 2018.0  10.0            1.0              0.0        1.0   
2                 2019.0   1.0            5.0              0.0        5.0   
3                 2019.0   3.0            2.0              0.0        2.0   
4                 2019.0   4.0            1.0              0.0        1.0   
..           ...     ...   ...            ...              ...        ...   
387      TELECON  2024.0   7.0            7.0             16.0       -9.0   
388      TELECON  2024.0   8.0            3.0              1.0        2.0   
389      TELECON  2024.0   9.0           12.0              0.0       12.0   
390      TELECON  2024.0  10.0           18.0             28.0      -10.0   
391      TELECON  2024.0  11.0           16.0              6.0       10.0   

     DIFERENCA_ACUMULADA  
0                    2.0  
1                    

In [ ]:
# Exportar o DataFrame para um arquivo Excel após limpar os caracteres ilegais
caminho_arquivo = 'resultado.xlsx'
resultado.to_excel(caminho_arquivo, index=False, sheet_name='resultado')

print(f"Arquivo Excel salvo em: {caminho_arquivo}")

Arquivo Excel salvo em: resultado.xlsx
